#### Download and extract preprocessed dataset from Google Drive

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%%time
!cp '/content/gdrive/My Drive/ML/dataset/LibriTTS/LibriNumpied2-train-test-vald.tar' .

CPU times: user 1.61 s, sys: 250 ms, total: 1.86 s
Wall time: 5min 28s


In [0]:
%%time
!tar xf '/content/LibriNumpied2-train-test-vald.tar'

CPU times: user 1.17 s, sys: 145 ms, total: 1.31 s
Wall time: 3min 55s


#### Install required dependencies

In [0]:
!pip install -q tensorflow==2.0.0-beta1 tensorflow-gpu==2.0.0-beta1 pydub webrtcvad

     |████████████████████████████████| 87.9MB 323kB/s 
     |████████████████████████████████| 348.9MB 56kB/s 
     |████████████████████████████████| 71kB 21.7MB/s 
     |████████████████████████████████| 3.1MB 38.6MB/s 
     |████████████████████████████████| 501kB 42.7MB/s 


#### Download code

In [0]:
!rm -rf SV2MTTS/

In [0]:
!git clone 'https://github.com/sagar-spkt/SV2MTTS'

Cloning into 'SV2MTTS'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 158 (delta 24), reused 24 (delta 10), pack-reused 118
Receiving objects: 100% (158/158), 50.59 KiB | 582.00 KiB/s, done.
Resolving deltas: 100% (96/96), done.


In [0]:
# Go to code directory
%cd SV2MTTS/

/content/SV2MTTS


#### Import basic library and set log directory and checkpoint variable

In [0]:
import glob
import shutil
import numpy as np
import pandas as pd

In [0]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.__version__

'2.0.0-beta1'

In [0]:
LOG_DIR = '/content/gdrive/My Drive/ML/voice_clone/training_logs/SV2MTTS'
MODEL_CHECKPOINT_NAME = 'model{epoch:03d}.hdf5'
MODEL_CHECKPOINT_DIRNAME = LOG_DIR + '/checkpoints/'
log_dir = './logs'  # Tensorboard
!mkdir -p '{MODEL_CHECKPOINT_DIRNAME}'

#### Tunnel tensorboard from localhost

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-08-01 10:39:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.111.123, 52.200.123.104, 52.54.237.49, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.111.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  13.3MB/s    in 1.0s    

2019-08-01 10:39:30 (13.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://9103f818.ngrok.io


#### Import models, data generator and other utilities

In [0]:
from models import get_full_model, get_speaker_embedding_model, get_synthesizer_model, load_saved_model
from data_loader import SynthesizerTrainGenerator
from training_utils import LearningRateSchedulerPerGlobalStep,lr_schedule_func, get_init_epoch
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
import hparams

#### Instantiate generators

In [0]:
train_gen = SynthesizerTrainGenerator('/content/content/LibriNumpied2-train-test-vald/train')
val_gen = SynthesizerTrainGenerator('/content/content/LibriNumpied2-train-test-vald/validation', num_buckets=5, shuffle=False)

#### Restore saved model if one exist in checkpoint directory otherwise create new model

In [0]:
init_epoch = get_init_epoch(MODEL_CHECKPOINT_DIRNAME)
if init_epoch:
    model_path = MODEL_CHECKPOINT_DIRNAME + MODEL_CHECKPOINT_NAME.format(epoch=init_epoch)
    model = load_saved_model(model_path)
else:
    model = get_synthesizer_model()
init_epoch

W0801 10:40:02.816275 140094351148928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


129

#### Callbacks

In [0]:
callbacks = [LearningRateSchedulerPerGlobalStep(lr_schedule_func, batches_per_epoch=len(train_gen), initial_epoch=init_epoch),
             TensorBoard(log_dir, update_freq='batch'),
             ModelCheckpoint(MODEL_CHECKPOINT_DIRNAME + MODEL_CHECKPOINT_NAME)]

#### Train

In [0]:
p = model.fit_generator(train_gen, epochs=500, validation_data=val_gen, callbacks=callbacks, initial_epoch=init_epoch)

Epoch 130/500
  63/1388 [>.............................] - ETA: 2:49:48 - loss: 0.1042 - decoder_loss: 0.0536 - postprocessing_loss: 0.0505